In [1]:
import os
import multiprocessing
#import multiprocessing as mp
from multiprocessing import Process, Manager, Pool, Queue
from itertools import islice
from collections import Counter
import re
import pandas as pd
import numpy as np
import glob
import nltk
from nltk import word_tokenize
from functools import reduce
from pathlib import Path
import shutil

In [2]:
nprocs = multiprocessing.cpu_count()
print(f"Number of CPU cores: {nprocs}")

Number of CPU cores: 16


In [3]:
# language pair
lang_folder = "Turkish"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> target language for learner
#lang_pair = "Arabic"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> native language

# source path
source_path = "/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Turkish/Deployment2/Result/Deploy2 Result Manuel/Turkish"

# parameter
file = "Basics Cover the Most Master File"  # file without extention
sheets = "85 Sentence"  # 2 gram target, 2 gram hybrid
file_ext = "85 Sentence"
first_column = 2  # start column location number
end_column = 3

# native word select for Part 2 
word_start = 0  # 0  # native word start index
word_end = 200 # 28  # native word end index

In [4]:
df_word_all = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Deployment/Data/Word/Word_Merge_Preprocess.xlsx")
df_word_all

,word,frequency
0,bir,18835735
1,bu,11062659
2,ne,8025880
3,ve,7766036
4,için,5484109
...,...,...
987924,karneleme,5
987925,karnaya,5
987926,dörtlümüzün,5
987927,karnavalınız,5


In [5]:
df_word_select = df_word_all.iloc[word_start:word_end,]
df_word_select.reset_index(inplace=True)
df_word_select

,index,word,frequency
0,0,bir,18835735
1,1,bu,11062659
2,2,ne,8025880
3,3,ve,7766036
4,4,için,5484109
...,...,...,...
195,195,saat,399989
196,196,onunla,399330
197,197,yapıyorsun,398274
198,198,neler,397377


In [11]:
df_file = pd.read_excel(f"{source_path}/{file}.xlsx", sheet_name=f"{sheets}")
df_file.sort_values("search_string",key=lambda x:x.str.len(), inplace=True, ascending=True)
df_file.reset_index(drop=True, inplace=True)
#df_file = df_file.head(1)
df_file

,search_string,start_time,end_time,sentence,video_id,video_url,word_index_list
0,şu adam şu adam,214,216,şu adam şu adam,mHkBxBLAZww,https://www.youtube.com/watch?v=mHkBxBLAZww&t=...,"[48, 81, 48, 81]"
1,ol başka biri ol,281,283,mark zuckerberg ol başka biri ol,30Sd_IpkgK4,https://www.youtube.com/watch?v=30Sd_IpkgK4&t=...,"[118, 71, 78, 118]"
2,en fazla iki saat,318,320,bu evde en fazla iki saat geçiririm,bt6fLjCmyvM,https://www.youtube.com/watch?v=bt6fLjCmyvM&t=...,"[44, 96, 110, 195]"
3,tam başka en uzun,361,366,o hep tam başka en uzun karbon zinciri simetrik,9MU9b0WWnNM,https://www.youtube.com/watch?v=9MU9b0WWnNM&t=...,"[112, 71, 44, 172]"
4,lütfen ilk önce onu,21,23,videoyu izlemediyseniz lütfen ilk önce onu izl...,DWIzzc0OaIM,https://www.youtube.com/watch?v=DWIzzc0OaIM&t=21s,"[58, 193, 51, 23]"
...,...,...,...,...,...,...,...
80,nasıl bir şekilde olduğunu ve neden o şekilde ...,14,17,şimdi apexin nasıl bir şekilde olduğunu ve ned...,owj1QZoYqFs,https://www.youtube.com/watch?v=owj1QZoYqFs&t=14s,"[39, 0, 181, 40, 3, 34, 6, 181, 40]"
81,çok iyi değil mi tam böyle gerçek gibi gerçek ...,3,6,çok iyi değil mi tam böyle gerçek gibi gerçek ...,OmT_BtZh8qU,https://www.youtube.com/watch?v=OmT_BtZh8qU&t=3s,"[9, 28, 14, 5, 112, 56, 183, 27, 183, 27, 60]"
82,her zaman öyle değil herkes için de her zaman ...,559,562,her zaman öyle değil herkes için de her zaman ...,cPoQ27hCNOU,https://www.youtube.com/watch?v=cPoQ27hCNOU&t=...,"[30, 35, 46, 14, 173, 4, 8, 30, 35, 46, 14]"
83,gerçekten uzun ve bu da benim için çok önemli ...,623,627,bu tabletin şarj ömrü batarya ömrü gerçekten u...,r_9ngjGh2Ks,https://www.youtube.com/watch?v=r_9ngjGh2Ks&t=...,"[70, 172, 3, 1, 15, 31, 4, 9, 156, 68, 67]"


In [7]:
def word_count_result(df,column_list): # df is dataframe, column_list is list value
    '''
    word_count_bool(df, column_list): df columns word count for word frequency\n
    df is dataframe, column_list is list value\n
    word_count_bool(df, ["word","twogram"]):
    '''
    list_all = []
    for i in df.loc[:,[x for x in column_list]].columns:
        var_list = df[f"{i}"].dropna().tolist()
        for j in var_list:
            list_all.append(j)
    text = " ".join(list_all)
    word_list = re.findall(r"\w+",text, re.UNICODE)
    df_word_list = pd.DataFrame(word_list, columns=["word"])
    #df_word_list.rename(columns={0:"word"}, inplace=True)
    df_word_count = pd.DataFrame(df_word_list.value_counts())
    df_word_count.reset_index(inplace=True)
    df_word_count.rename(columns={0:"word_count"}, inplace=True)
    
    return  df_word_count

In [12]:
df_file_count = word_count_result(df_file, df_file.columns[0:1])
df_file_count.sort_values(by="word_count", ascending=False)
df_file_count

,word,word_count
0,bir,15
1,çok,14
2,güzel,8
3,bakalım,8
4,değil,8
...,...,...
195,ister,1
196,hiç,1
197,hey,1
198,herkes,1


In [13]:
df_check = df_file_count[df_file_count["word_count"] > 3]
df_check

,word,word_count
0,bir,15
1,çok,14
2,güzel,8
3,bakalım,8
4,değil,8
5,için,7
6,mi,7
7,çünkü,7
8,tamam,7
9,ne,6


In [14]:
check_list = df_check["word"].to_list()
check_list

['bir',
 'çok',
 'güzel',
 'bakalım',
 'değil',
 'için',
 'mi',
 'çünkü',
 'tamam',
 'ne',
 'kadar',
 'buraya',
 'böyle',
 'yani',
 'şey',
 'bu',
 'olsun',
 've',
 'oluyor',
 'var',
 'biraz',
 'şimdi',
 'daha',
 'gerçekten',
 'önce',
 'musun',
 'gibi',
 'geliyor',
 'gel',
 'ya',
 'zaman',
 'en',
 'ona',
 'hemen',
 'şekilde',
 'peki',
 'tam',
 'ben',
 'tekrar',
 'ama',
 'yok',
 'her',
 'aynı',
 'birlikte',
 'biri']

In [15]:
test = ["aet","önce"]

In [16]:
test in check_list

False

In [43]:
for var_sent in df_file["search_string"]:
    result_list = []
    words = re.findall(r"\w+",var_sent, re.UNICODE)
    for word in words:
        if word in check_list:
            result_list.append(True)
        else:
            result_list.append(False)
    if all(result_list):
        df_file = df_file[df_file["search_string"] != f"{var_sent}"]
        df_file_count = word_count_result(df_file, df_file.columns[0:1])
        df_file_count.sort_values(by="word_count", ascending=False)
        df_check = df_file_count[df_file_count["word_count"] > 3]
        check_list = df_check["word"].to_list()
    else:
        pass

In [17]:
df_file[df_file["search_string"] != "şu adam şu adam"]

,search_string,start_time,end_time,sentence,video_id,video_url,word_index_list
1,ol başka biri ol,281,283,mark zuckerberg ol başka biri ol,30Sd_IpkgK4,https://www.youtube.com/watch?v=30Sd_IpkgK4&t=...,"[118, 71, 78, 118]"
2,en fazla iki saat,318,320,bu evde en fazla iki saat geçiririm,bt6fLjCmyvM,https://www.youtube.com/watch?v=bt6fLjCmyvM&t=...,"[44, 96, 110, 195]"
3,tam başka en uzun,361,366,o hep tam başka en uzun karbon zinciri simetrik,9MU9b0WWnNM,https://www.youtube.com/watch?v=9MU9b0WWnNM&t=...,"[112, 71, 44, 172]"
4,lütfen ilk önce onu,21,23,videoyu izlemediyseniz lütfen ilk önce onu izl...,DWIzzc0OaIM,https://www.youtube.com/watch?v=DWIzzc0OaIM&t=21s,"[58, 193, 51, 23]"
5,diye söyle bak hala,104,106,o taşlarını gecesi zor için yoğurduktan sonra ...,P5bYtFrqw68,https://www.youtube.com/watch?v=P5bYtFrqw68&t=...,"[123, 161, 64, 167]"
...,...,...,...,...,...,...,...
80,nasıl bir şekilde olduğunu ve neden o şekilde ...,14,17,şimdi apexin nasıl bir şekilde olduğunu ve ned...,owj1QZoYqFs,https://www.youtube.com/watch?v=owj1QZoYqFs&t=14s,"[39, 0, 181, 40, 3, 34, 6, 181, 40]"
81,çok iyi değil mi tam böyle gerçek gibi gerçek ...,3,6,çok iyi değil mi tam böyle gerçek gibi gerçek ...,OmT_BtZh8qU,https://www.youtube.com/watch?v=OmT_BtZh8qU&t=3s,"[9, 28, 14, 5, 112, 56, 183, 27, 183, 27, 60]"
82,her zaman öyle değil herkes için de her zaman ...,559,562,her zaman öyle değil herkes için de her zaman ...,cPoQ27hCNOU,https://www.youtube.com/watch?v=cPoQ27hCNOU&t=...,"[30, 35, 46, 14, 173, 4, 8, 30, 35, 46, 14]"
83,gerçekten uzun ve bu da benim için çok önemli ...,623,627,bu tabletin şarj ömrü batarya ömrü gerçekten u...,r_9ngjGh2Ks,https://www.youtube.com/watch?v=r_9ngjGh2Ks&t=...,"[70, 172, 3, 1, 15, 31, 4, 9, 156, 68, 67]"


In [49]:
df_file.drop

0

In [44]:
result_list

[False, False, False, False]

In [45]:
all(result_list)

False

In [37]:
a = []

In [38]:
a.append(True)

In [40]:
a.append(False)

In [41]:
a

[True, False]